In [1]:
import keras.utils
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten

inp = Input(shape=(240, 320, 1))

n_filters = 512
max_p = inp

for _ in range(3):
    conv = Conv2D(filters=n_filters, kernel_size=5, activation='relu')(max_p)
    max_p = MaxPooling2D(pool_size=(2, 2))(conv)
    
    n_filters //= 2

conv = Conv2D(filters=n_filters, kernel_size=3, activation='relu')(max_p)
max_p = MaxPooling2D(pool_size=(2, 2))(conv)

flatten = Flatten()(max_p)
dense = Dense(128, activation='relu')(flatten)
# out = Dense(14, activation='relu')(dense)

head_out = Dense(1, activation='relu', name='head_out')(dense)
neck_out = Dense(1, activation='relu', name='neck_out')(dense)
shoulder_to_shoulder_out = Dense(1, activation='relu', name='shoulder_to_shoulder_out')(dense)
arm_out = Dense(1, activation='relu', name='arm_out')(dense)
shoulder_to_wrist_out = Dense(1, activation='relu', name='shoulder_to_wrist_out')(dense)
torso_out = Dense(1, activation='relu', name='torso_out')(dense)
bicep_out = Dense(1, activation='relu', name='bicep_out')(dense)
wrist_out = Dense(1, activation='relu', name='wrist_out')(dense)
chest_out = Dense(1, activation='relu', name='chest_out')(dense)
waist_out = Dense(1, activation='relu', name='waist_out')(dense)
pelvis_out = Dense(1, activation='relu', name='pelvis_out')(dense)
leg_out = Dense(1, activation='relu', name='leg_out')(dense)
inner_leg_out = Dense(1, activation='relu', name='inner_leg_out')(dense)
thigh_out = Dense(1, activation='relu', name='thigh_out')(dense)
knee_out = Dense(1, activation='relu', name='knee_out')(dense)
calf_out = Dense(1, activation='relu', name='calf_out')(dense)


model = Model(
    inputs=inp,
    outputs=[
    head_out,
    neck_out,
    shoulder_to_shoulder_out,
    arm_out,
    shoulder_to_wrist_out,
    torso_out,
    bicep_out,
    wrist_out,
    chest_out,
    waist_out,
    pelvis_out,
    leg_out,
    inner_leg_out,
    thigh_out,
    knee_out,
    leg_out,
    inner_leg_out,
    thigh_out,
    calf_out
    ],
    name='conv_bodies'
)

model.summary()

Model: "conv_bodies"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 240, 320,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 236, 316,  │     13,312 │ input_layer[0][0] │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 118, 158,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 114, 154,  │  3,277,056 │ max_pooling2d[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 57, 77,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 53, 73,    │    819,328 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 26, 36,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 24, 34,    │     73,792 │ max_pooling2d_2[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 12, 17,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 13056)     │          0 │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │  1,671,296 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ head_out (Dense)    │ (None, 1)         │        129 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ neck_out (Dense)    │ (None, 1)         │        129 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shoulder_to_should… │ (None, 1)         │        129 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ arm_out (Dense)     │ (None, 1)         │        129 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shoulder_to_wrist_… │ (None, 1)         │        129 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ torso_out (Dense)   │ (None, 1)         │        129 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bicep_out (Dense)   │ (None, 1)         │        129 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ wrist_out (Dense)   │ (None, 1)         │        129 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 5,856,848 (22.34 MB)

 Trainable params: 5,856,848 (22.34 MB)

 Non-trainable params: 0 (0.00 B)

In [2]:
# from keras.utils import plot_model
# 
# plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, show_layer_activations=True)

In [3]:
#FOR BODY_M DATASET
# import csv
# 
# with open('../bodym-dataset/train/subject_to_photo_map.csv', 'r') as f:
#     reader = csv.reader(f)
#     uuid_raw = list(reader)
# from collections import defaultdict
# 
# uuid_map = defaultdict(list)
# 
# for subject_id, photo_id in uuid_raw[1:]:
#     uuid_map[subject_id].append(photo_id)
# image_data = {}
# 
# with open('../bodym-dataset/train/measurements.csv', 'r') as f:
#     reader = csv.reader(f)
#     
#     for line in reader:
#         subject_id = line[0]
#         
#         
#         for image_id in uuid_map[subject_id]:
#             image_data[image_id] = np.array(list(map(float, line[1:])))
#             
# import os
# 
# import cv2
# from tqdm import tqdm
# 
# raw_dataset = []
# 
# for image_id in tqdm(image_data):
#     img = cv2.imread(os.path.join('../bodym-dataset/train/mask', image_id + '.png'), cv2.IMREAD_GRAYSCALE)
#     img = cv2.resize(img, (180, 240))
#     
#     raw_dataset.append((
#         np.expand_dims(img, 2), 
#         image_data[image_id])
#     )

In [4]:
import cv2
import os
from tqdm import tqdm

avatar_count = 79999
measurement_data = {}

train_data = keras.utils.image_dataset_from_directory(
    directory="../Surreact-APose/train/imgs_nobg_frontEDITED/",
    labels=None,
    batch_size=32,
    image_size=(240, 320),
    shuffle=True,
    validation_split=0.2,
    color_mode="grayscale",
    subset="both",
    seed=69
)

for number in tqdm(range(avatar_count)):
    if number == 5:
        continue
#     source_image = cv2.imread(os.path.join(f"../Surreact-APose/train/imgs_nobg_front/Avatar_{number:06d}.png"), cv2.IMREAD_GRAYSCALE)
#     silhouette = (source_image > 1).astype(np.uint8)
    measurement_data[number] = np.load(f"../Surreact-APose/train/measurements/Avatar_{number:06d}.npy")

2024-03-18 20:03:13.749053: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 20:03:15.024570: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 79998 files.
Using 63999 files for training.
Using 15999 files for validation.


2024-03-18 20:03:18.476986: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-18 20:03:18.481033: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-18 20:03:18.481261: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
y_true = np.array([*measurement_data.values()])

In [9]:
(
    y_true_head, y_true_neck, y_true_sts,y_true_arm, y_true_stw, y_true_torso, y_true_bicep, y_true_wrist, y_true_chest, y_true_waist, y_true_pelvis, y_true_leg, y_true_inner_leg, y_true_thigh, y_true_knee, y_true_calf, height
) = np.hsplit(y_true, 17) #17 is height

In [11]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import tensorflow as tf


tf.data.Dataset.zip(train_data[0], y_true)

model.compile(
    optimizer=Adam(),
    loss={
        'head_out': 'mse',
        'neck_out': 'mse',
        'shoulder_to_shoulder_out': 'mse',
        'arm_out': 'mse',
        'shoulder_to_wrist_out': 'mse',
        'torso_out': 'mse',
        'bicep_out': 'mse',
        'wrist_out': 'mse',
        'chest_out': 'mse',
        'waist_out': 'mse',
        'pelvis_out': 'mse',
        'leg_out': 'mse',
        'inner_leg_out': 'mse',
        'thigh_out': 'mse',
        'knee_out': 'mse',
        'calf_out': 'mse'
    },
    metrics={
        'head_out': ['mae'],
        'neck_out': ['mae'],
        'shoulder_to_shoulder_out': ['mae'],
        'arm_out': ['mae'],
        'shoulder_to_wrist_out': ['mae'],
        'torso_out': ['mae'],
        'bicep_out': ['mae'],
        'wrist_out': ['mae'],
        'chest_out': ['mae'],
        'waist_out': ['mae'],
        'pelvis_out': ['mae'],
        'leg_out': ['mae'],
        'inner_leg_out': ['mae'],
        'thigh_out': ['mae'],
        'knee_out': ['mae'],
        'calf_out': ['mae']
    },
)




model.fit(
    x=train_data[0],
    y={
        'head_out': y_true_head,
        'neck_out': y_true_neck,
        'shoulder_to_shoulder_out': y_true_sts,
        'arm_out': y_true_arm,
        'shoulder_to_wrist_out': y_true_stw,
        'torso_out': y_true_torso,
        'bicep_out': y_true_bicep,
        'wrist_out': y_true_wrist,
        'chest_out': y_true_chest,
        'waist_out': y_true_waist,
        'pelvis_out': y_true_pelvis,
        'leg_out': y_true_leg,
        'inner_leg_out': y_true_inner_leg,
        'thigh_out': y_true_thigh,
        'knee_out': y_true_knee,
        'calf_out': y_true_calf
    },
    batch_size=8,
    epochs=10,
    validation_data=train_data[1],
    shuffle=True,
    callbacks=[TensorBoard(write_graph=False)]
)

ValueError: Slicing dataset elements is not supported for rank 0.

In [17]:
for element in train_data[0].take(1):  # Take one batch from the dataset
    for item in element:
        print("Rank:", tf.rank(item))

Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype=int32)
Rank: tf.Tensor(3, shape=(), dtype

2024-03-18 22:00:22.075027: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
model.save('models/abcd.keras')

In [ ]:
from keras.models import load_model

model = load_model('')

In [ ]:
# import os
# from tqdm import tqdm
# 
# avatar_count = 79999
# 
# for number in tqdm(range(avatar_count)):
#     if number == 5:
#         continue
#     file = (tuple(np.load(f"../Surreact-APose/train/bodymeasurements/Avatar_{number:06d}.npy", allow_pickle=True)))
#     measurements = []
#     for value in file:
#         measurements.append(float(value))
#     np.save(f"../Surreact-APose/train/measurements/Avatar_{number:06d}", measurements)        